# 🚀 ELS GPU 테스트 - Google Drive 방식

**가장 간단한 방법!**

---

## ⚡ Step 0: GPU 활성화

1. `Runtime` → `Change runtime type`
2. Hardware accelerator: **GPU**
3. Save


## 📁 Step 1: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive 연결 완료!")

## 📤 Step 2: 프로젝트 파일 업로드

**로컬 컴퓨터에서:**

1. Google Drive 열기 (drive.google.com)
2. `els-fdm-pricer-colab.tar.gz` 파일을 드래그 앤 드롭으로 업로드
3. 업로드 완료 후 아래 셀 실행

In [ ]:
# Drive에서 파일 복사 및 압축 해제
!cp /content/drive/MyDrive/els-fdm-pricer-colab.tar.gz .
!tar -xzf els-fdm-pricer-colab.tar.gz

print("✓ 프로젝트 파일 준비 완료!")

## 🔧 Step 3: 패키지 설치

In [ ]:
# CuPy 설치
!pip install -q cupy-cuda12x numpy scipy matplotlib

print("✓ 패키지 설치 완료!")

## 🎮 Step 4: GPU 확인

In [ ]:
!nvidia-smi

import cupy as cp
print(f"\n✓ GPU: {cp.cuda.Device().name}")
print(f"✓ GPU 메모리: {cp.cuda.Device().mem_info[1] / 1024**3:.1f} GB")

## 🧪 Step 5: GPU 테스트

In [ ]:
import sys
sys.path.insert(0, '/content/els-fdm-pricer')

from src.models.els_product import create_sample_els
from src.pricing.gpu_els_pricer import price_els_gpu
from src.pricing.els_pricer import price_els
import time

print("="*80)
print("GPU vs CPU 벤치마크")
print("="*80)

product = create_sample_els()

# CPU 테스트
print("\n[CPU] 100×100×200 계산 중...", end="", flush=True)
start = time.time()
result_cpu = price_els(product, N1=100, N2=100, Nt=200, verbose=False)
time_cpu = time.time() - start
print(f" {time_cpu:.2f}초")

# GPU 테스트
print("[GPU] 100×100×200 계산 중...", end="", flush=True)
start = time.time()
result_gpu = price_els_gpu(product, N1=100, N2=100, Nt=200, use_gpu=True, verbose=False)
time_gpu = time.time() - start
print(f" {time_gpu:.2f}초")

speedup = time_cpu / time_gpu

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"CPU 시간: {time_cpu:.2f}초")
print(f"GPU 시간: {time_gpu:.2f}초")
print(f"속도 향상: {speedup:.1f}배")
print(f"가격 차이: {abs(result_cpu['price'] - result_gpu['price']):.6f}")
print("="*80)

if speedup > 10:
    print("\n🚀 GPU 매우 빠름!")
else:
    print("\n✓ GPU 효과 확인!")

## 🎯 Step 6: 대규모 테스트 (200×200×1000)

In [ ]:
print("="*80)
print("대규모 그리드 GPU 테스트 (200×200×1000)")
print("="*80)

print("\n계산 중...", end="", flush=True)
start = time.time()
result = price_els_gpu(product, N1=200, N2=200, Nt=1000, use_gpu=True, verbose=False)
elapsed = time.time() - start
print(" 완료!")

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"ELS 가격: {result['price']:.4f}")
print(f"GPU 시간: {elapsed:.2f}초")
print(f"처리량: {40_000_000/elapsed:,.0f} points/sec")
print()
print("CPU 시간 (측정값): 78.26초")
print(f"속도 향상: {78.26/elapsed:.1f}배")
print("="*80)

if elapsed < 2.0:
    print("\n🎉 실시간 프라이싱 가능!")